In [1]:
import spacy
from spacy import displacy
import numpy as np
import pandas as pd
import re

In [2]:
nlp = spacy.load('en_core_web_sm')

In [3]:
# from google.colab import files


# uploaded = files.upload()


In [4]:
df = pd.read_csv('heuristics.csv',header=None)
print(df)


                                                    0
0   Where immediately before the issue of a notifi...
1   Where in respect of an industrial dispute rela...
2   At the time of hearing, the authority may dire...
3   The authority referred to in sub-section (6) m...
4   Finality of orders constituting Board, Committ...
5   Chief Inspector- The State Government may, by ...
6   Where the applicant succeeds in such proceedin...
7   Penalty for obstructing Inspector. -Whoever wi...
8   Protection to persons acting under the Act. - ...
9   Transfer of cases from one Court to another. -...
10  Removal of difficulties.-If any difficulty ari...
11  Provided that the appropriate Government may, ...
12  Provided that in relation to the State of Jamm...
13  Provided further that when the provisions of t...
14  Every employee shall be entitled to be paid by...
15  Subject to the other provisions of this Act, e...
16  Where in respect of any accounting year referr...
17  In computing the allocab

In [5]:
df.head()

,0
0,Where immediately before the issue of a notifi...
1,Where in respect of an industrial dispute rela...
2,"At the time of hearing, the authority may dire..."
3,The authority referred to in sub-section (6) m...
4,"Finality of orders constituting Board, Committ..."


In [6]:
i = 3
for index,row in df.iterrows():
    sentence = row.values[0]
    doc = nlp(sentence)
    print(i,"\t")
    for token in doc:
        ancestors = [t.text for t in token.ancestors]
        children = [t.text for t in token.children]
        if(token.dep_ == "ROOT"):
            print(token.text, "\t", 
          token.pos_, "\t" )
    i = i + 1
    print("\n")

3 	
be 	 AUX 	


4 	
apply 	 VERB 	


5 	
direct 	 VERB 	
paid 	 VERB 	


6 	
allow 	 VERB 	


7 	
Finality 	 NOUN 	
called 	 VERB 	


8 	
appoint 	 VERB 	


9 	
calculate 	 VERB 	
be 	 AUX 	


10 	
Penalty 	 NOUN 	
obstructs 	 VERB 	


11 	
Protection 	 NOUN 	
lie 	 VERB 	


12 	
Transfer 	 NOUN 	
- 	 PUNCT 	
subordinate 	 ADJ 	


13 	
do 	 VERB 	


14 	
Provided 	 VERB 	
employing 	 VERB 	


15 	
Provided 	 VERB 	


16 	
Provided 	 VERB 	


17 	
entitled 	 VERB 	


18 	
bound 	 VERB 	


19 	
bound 	 VERB 	


20 	
taken 	 VERB 	


21 	
calculated 	 VERB 	


22 	
reduced 	 VERB 	


23 	
exceeds 	 VERB 	
carried 	 VERB 	


24 	
carried 	 VERB 	


25 	
apply 	 VERB 	


26 	
taken 	 VERB 	


27 	
Where 	 SCONJ 	


28 	
be 	 AUX 	


29 	
apply 	 VERB 	


30 	
be 	 AUX 	


31 	
sells 	 VERB 	


32 	
make 	 VERB 	


33 	
Provided 	 VERB 	


34 	
Provided 	 VERB 	


35 	
deemed 	 VERB 	


36 	
14 	 NUM 	
[ 	 X 	
referred 	 VERB 	


37 	
comply 	 VERB 	


38 	
specified 	 VERB 	


39 	
enable 

In [7]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    if(token.dep_ == "ROOT"):
        print(token.text, "\t", token.i, "\t", 
          token.pos_, "\t", token.dep_, "\t", 
          ancestors, "\t", children)

take 	 3 	 VERB 	 ROOT 	 [] 	 ['court', 'shall', 'cognizance', ',', 'save']
authorised 	 76 	 VERB 	 ROOT 	 [] 	 ['[', 'specially', 'in', 'by']


In [8]:
sentence = "Protection to persons acting under the Act. - No suit, prosecution or other legal proceeding whatsoever shall lie against any person for anything which is in good faith done or intended to be done in due discharge of his duties under this Act."

In [9]:
doc = nlp(sentence)

In [10]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    children = [t.text for t in token.children]
    if(token.dep_ == "ROOT"):
        print(token.text, "\t", token.i, "\t", 
            token.pos_, "\t", token.dep_, "\t", 
            ancestors, "\t", children)

Protection 	 0 	 NOUN 	 ROOT 	 [] 	 ['to', '.', '-']
lie 	 19 	 VERB 	 ROOT 	 [] 	 ['suit', 'shall', 'against', 'for', '.']


In [11]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [12]:
root_token = find_root_of_sentence(doc)

In [13]:
def find_other_verbs(doc, root_token):
    other_verbs = []
    for token in doc:
        ancestors = list(token.ancestors)
        if (token.pos_ == "VERB" and len(ancestors) == 1\
            and ancestors[0] == root_token):
            other_verbs.append(token)
    return other_verbs

In [14]:
other_verbs = find_other_verbs(doc, root_token)

In [15]:
def get_clause_token_span_for_verb(verb, doc, all_verbs):
    first_token_index = len(doc)
    last_token_index = 0
    this_verb_children = list(verb.children)
    for child in this_verb_children:
        if (child not in all_verbs):
            if (child.i < first_token_index):
                first_token_index = child.i
            if (child.i > last_token_index):
                last_token_index = child.i
    return(first_token_index, last_token_index)

In [16]:
token_spans = []   
all_verbs = [root_token] + other_verbs
for other_verb in all_verbs:
    (first_token_index, last_token_index) = \
     get_clause_token_span_for_verb(other_verb, 
                                    doc, all_verbs)
    token_spans.append((first_token_index, 
                        last_token_index))

In [17]:
sentence_clauses = []
for token_span in token_spans:
    start = token_span[0]
    end = token_span[1]
    if (start < end):
        clause = doc[start:end]
        sentence_clauses.append(clause)
sentence_clauses = sorted(sentence_clauses, 
                          key=lambda tup: tup[0])

In [18]:
clauses_text = [clause.text for clause in sentence_clauses]
print(clauses_text)

['suit, prosecution or other legal proceeding whatsoever shall lie against any person for anything which is in good faith done or intended to be done in due discharge of his duties under this Act']


# **REPLICATING ONLINE SPACY**

DHRUV'S AIM-ATTRIBUTE-DEONTIC

In [19]:
nlp = spacy.load("en_core_web_sm")
sentence = "Provided that nothing contained in this sub-section shall render any such person liable to any punishment if he proves that the offence was committed without his knowledge or that he exercised all; due diligence to prevent the commission of such offence"
doc = nlp(sentence)


def merge_phrases(doc):
    with doc.retokenize() as retokenizer:
        for np in list(doc.noun_chunks):
            attrs = {
                "tag": np.root.tag_,
                "lemma": np.root.lemma_,
                "ent_type": np.root.ent_type_,
            }
            retokenizer.merge(np, attrs=attrs)
    return doc

def merge_punct(doc):
    spans = []
    for word in doc[:-1]:
        if word.is_punct or not word.nbor(1).is_punct:
            continue
        start = word.i
        end = word.i + 1
        while end < len(doc) and doc[end].is_punct:
            end += 1
        span = doc[start:end]
        spans.append((span, word.tag_, word.lemma_, word.ent_type_))
    with doc.retokenize() as retokenizer:
        for span, tag, lemma, ent_type in spans:
            attrs = {"tag": tag, "lemma": lemma, "ent_type": ent_type}
            retokenizer.merge(span, attrs=attrs)
    return doc

# Merge noun phrases into one token.
doc = merge_phrases(doc)
# Attach punctuation to tokens
doc = merge_punct(doc)

In [20]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    ancestors_token = [t for t in token.ancestors]
    children = [t.text for t in token.children]
    children_token = [t for t in token.children]
    left = [t.text for t in token.lefts]
    right = [t.text for t in token.rights]
    print(token.text, token.pos_, token.dep_,ancestors,children,left,right)


Provided VERB prep ['diligence'] ['render'] [] ['render']
that SCONJ mark ['render', 'Provided', 'diligence'] [] [] []
nothing PRON nsubj ['render', 'Provided', 'diligence'] ['contained'] [] ['contained']
contained VERB acl ['nothing', 'render', 'Provided', 'diligence'] ['in'] [] ['in']
in ADP prep ['contained', 'nothing', 'render', 'Provided', 'diligence'] ['this sub-'] [] ['this sub-']
this sub- NOUN pobj ['in', 'contained', 'nothing', 'render', 'Provided', 'diligence'] [] [] []
section NOUN nsubj ['render', 'Provided', 'diligence'] [] [] []
shall AUX aux ['render', 'Provided', 'diligence'] [] [] []
render VERB ccomp ['Provided', 'diligence'] ['that', 'nothing', 'section', 'shall', 'liable', 'proves'] ['that', 'nothing', 'section', 'shall'] ['liable', 'proves']
any such person NOUN nsubj ['liable', 'render', 'Provided', 'diligence'] [] [] []
liable ADJ ccomp ['render', 'Provided', 'diligence'] ['any such person', 'to'] ['any such person'] ['to']
to ADP prep ['liable', 'render', 'Prov

In [21]:
ners = [(e,type(e)) for e in doc.ents]
ners

[]

In [22]:
nlp.pipeline
# Tagger identifies verb, noun etc
# Parser identifies relationships b/w different tokens
# ner labels named entities

[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x203cb06fe80>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x203cb06ef80>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x203cb0a52a0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x203cc360c40>),
 ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer at 0x203cc3276c0>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x203cb0a50e0>)]

In [23]:
# attribute = []
# attribute_token = []
# aim = []
# aim_token = []
# deontic = []
# deontic_token = []
# aim_2 = []
# aim_2_token = []


For Attribute

In [24]:
# i = 0
# for entity in doc:
#     print(entity)
#     if ((entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or "PRON") and (entity.dep_ == "nsubjpass" or entity.dep_ == "nsubj")):
#         attribute.append(entity.text)
#         attribute_token.append(entity)
#         for attr in entity.children:
#             if attr.dep_ == 'conj' and (attr.pos_ == "PROPN" or attr.pos_ == "NOUN"):
#                 attribute.append(attr.text)
#                 attribute_token.append(attr)
    
#     if(len(attribute)>0):
#         break

For Deontic

In [25]:
# for entity in doc:
#     print(entity)
#     if ((entity.pos_ == "AUX") and (entity.dep_ == "aux" or entity.dep_ == "auxpass")):
#         deontic.append(entity.text)
#         deontic_token.append(entity)
#         break
  
  

For Aim

In [26]:
# for entity in doc:
#   if (entity.pos_ =='NOUN' or entity.pos_ =='VERB'):
#     for child in entity.children:
#       if (child in attribute_token) and (child.dep_ == 'nsubj' or child.dep_ =='nsubjpass'):
#         aim.append(entity.text)
#         aim_token.append(entity)

#         for attr in entity.children:
#           if attr.dep_ == 'conj' and (attr.pos_ == "NOUN" or attr.pos_ == "VERB"):
#             aim.append(attr.text)
#             aim_token.append(attr)
  
#   if(len(aim)>0):
#     break

Aim Updated code

In [27]:
# for entity in doc:
#   if (entity.pos_ =='VERB'):
#     for child in entity.children:
#       if (child in deontic_token) and (child.dep_ == 'aux'):
#         aim_2.append(entity.text)
#         aim_2_token.append(entity)

#         for attr in entity.children:
#           if attr.dep_ == 'conj' and (attr.pos_ == "NOUN" or attr.pos_ == "VERB"):
#             aim.append(attr.text)
#             aim_token.append(attr)
  
#   if(len(aim_2)>0):
#     break

## Heuristic 2


In [28]:
sentence = "Where immediately before the issue of a notification under section 5 fixing or revising the minimum rates of wages in respect of any scheduled employment, wages at a rate higher than the rate so fixed or revised, were payable either by contract or agreement, or, under any other law for the time being in force, then, notwithstanding anything contained in this Act, wages at such higher rate shall be payable to the employees in such scheduled employment and the wages so payable shall be deemed to be the minimum wages for the purposes of this Act."
doc = nlp(sentence)

In [29]:
# Merge noun phrases into one token.
doc = merge_phrases(doc)
# Attach punctuation to tokens
doc = merge_punct(doc)

In [30]:
root_tokens = []

In [31]:
for token in doc:
    ancestors = [t.text for t in token.ancestors]
    ancestors_token = [t for t in token.ancestors]
    children = [t.text for t in token.children]
    children_token = [t for t in token.children]
    left = [t.text for t in token.lefts]
    right = [t.text for t in token.rights]
    if(token.dep_ == "ROOT"):
        root_tokens.append(token)
        print(token.text, "\t", 
          token.pos_, "\t" )
        print("\n")
    print(token.text, token.pos_, token.dep_,ancestors,children)


Where SCONJ advmod ['were', 'be'] []
immediately ADV advmod ['were', 'be'] []
before ADP prep ['were', 'be'] ['the issue']
the issue NOUN pobj ['before', 'were', 'be'] ['of']
of ADP prep ['the issue', 'before', 'were', 'be'] ['a notification']
a notification NOUN pobj ['of', 'the issue', 'before', 'were', 'be'] ['under', 'fixing']
under ADP prep ['a notification', 'of', 'the issue', 'before', 'were', 'be'] ['section']
section NOUN pobj ['under', 'a notification', 'of', 'the issue', 'before', 'were', 'be'] ['5']
5 NUM nummod ['section', 'under', 'a notification', 'of', 'the issue', 'before', 'were', 'be'] []
fixing NOUN acl ['a notification', 'of', 'the issue', 'before', 'were', 'be'] ['or', 'revising']
or CCONJ cc ['fixing', 'a notification', 'of', 'the issue', 'before', 'were', 'be'] []
revising VERB conj ['fixing', 'a notification', 'of', 'the issue', 'before', 'were', 'be'] ['the minimum rates', 'in']
the minimum rates NOUN dobj ['revising', 'fixing', 'a notification', 'of', 'the is

In [32]:
root_tokens

[be]

In [33]:
root_verb = []
root_aux = []
root_noun = []
root_others = []
case = 3

for token in root_tokens:
    if(token.pos_ == 'VERB'):
        root_verb.append(token)
    elif(token.pos_ == 'AUX'):
        root_aux.append(token)
    elif(token.pos_ == 'NOUN'):
        root_noun.append(token)
    else:
        root_others.append(token)

if(len(root_verb)>0):
    case = 1
elif(len(root_aux)>0):
    case = 2
elif(len(root_noun)>0):
    case = 3
elif(len(root_others)>0):
    case = 3
    
print(case)

2


In [34]:
def find_case(doc):
    root_tokens = []
    for token in doc:
        ancestors = [t.text for t in token.ancestors]
        ancestors_token = [t for t in token.ancestors]
        children = [t.text for t in token.children]
        children_token = [t for t in token.children]
        left = [t.text for t in token.lefts]
        right = [t.text for t in token.rights]
        if(token.dep_ == "ROOT"):
            root_tokens.append(token)
    #         print(token.text, "\t", 
    #           token.pos_, "\t" )
    #         print("\n")
    #     print(token.text, token.pos_, token.dep_,ancestors,children)
    root_verb = []
    root_aux = []
    root_noun = []
    root_others = []
    case = 3

    for token in root_tokens:
        if(token.pos_ == 'VERB'):
            root_verb.append(token)
        elif(token.pos_ == 'AUX'):
            root_aux.append(token)
        elif(token.pos_ == 'NOUN'):
            root_noun.append(token)
        else:
            root_others.append(token)

    if(len(root_verb)>0):
        case = 1
    elif(len(root_aux)>0):
        case = 2
    elif(len(root_noun)>0):
        case = 3
    elif(len(root_others)>0):
        case = 3
    print(f"\n Case = {case}")
    return case, root_verb,root_aux

In [35]:
def tokenise(case,doc,root_verb,root_aux):
    
    aim_token = []
    attr_token = []
    deontic_token = []
    
    keywords = ["shall","may","must"]
    secondary_keywords = ["is","has","have","was","having"]
    
    
    if(case==1):
        #AIM
        #working on root_verb
        flag = 0
        #first adding all VERBS with conj and ccomp to the root_verb list
        for token in root_verb:
            for children in token.children:
                if(children.dep_ == 'ccomp' or children.dep_ == 'conj'):
                    root_verb.append(children)

        #in GIVEN ORDER in the list, if we find a token with both nsubj and aux then we return it
        for token in root_verb:
            children_dep = [child.dep_ for child in token.children]
            str1 = "nsubj"
            str2 = "aux"

            if(str1 in children_dep and str2 in children_dep):
                aim_token.append(token)
                flag = 1
                break #only 1 aim in this case
        #nusbjpass and aux if not aux and nsubj    
        #if we dont find any such VERB then we take entire root_verb as aim (some of them might be right as there could be multiple)
        if(flag == 0):
            aim_token = root_verb
        #NOTE: AIM never empty since CASE 1
        
        #DEONTIC
        #deontic won't be empty due to keywords and secondary keywords
        for token in aim_token:
            for child in token.children:
                if(child.dep_ =='aux' and child.pos_ == 'AUX'):
                    deontic_token.append(child)
                    break
            if(len(deontic_token)>0):
                break

        #if no such deontic found then search in keywords and later in secondary keywords
        if(len(deontic_token) == 0):
            for keyword in keywords:
                for token in doc:
                    if(keyword in token.text and token.pos_ =="AUX"):
                        deontic_token.append(token)
                        break
                if(len(deontic_token)>0):
                    break

        if(len(deontic_token) == 0):
            for keyword in secondary_keywords:
                for token in doc:
                    if(keyword in token.text and token.pos_ =="AUX"):
                        deontic_token.append(token)
                        break
                if(len(deontic_token)>0):
                    break
                    
        #ATTRIBUTE
        #first check for nsubj
        for token in aim_token:
            for entity in token.children:
                if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                    attr_token.append(entity)

        #else check for nsubpass
        if(len(attr_token) == 0):
            for token in aim_token:
                for entity in token.children:
                    if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                        attr_token.append(entity)

        #else if empty then check in entire sentence
        if(len(attr_token) == 0):
            for token in doc:
                for entity in token.children:
                    if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                        attr_token.append(entity)
                        break
                if(len(attr_token) > 0):
                    break

        if(len(attr_token) == 0):
            for token in doc:
                for entity in token.children:
                    if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                        attr_token.append(entity)
                        break
                if(len(attr_token) > 0):
                    break
                    
                    
                    
    elif(case==2):
        #AIM
        for token in root_aux:
            for children in token.children:
                if((children.dep_ == 'ccomp' or children.dep_ == 'conj') and children.pos_ == 'VERB'):
                    aim_token.append(children)

        if(len(aim_token) == 0):
            for children in token.children:
                if((children.dep_ == 'advcl') and children.pos_ == 'NOUN'):
                    aim_token.append(children)
            for token in aim_token:
                for children in token.children:
                    if((children.dep_ == 'ccomp' or children.dep_ == 'conj') and children.pos_ == 'VERB'):
                        aim_token.append(children)
                        
        if(len(aim_token) == 0):
            for token in root_aux:
                for children in token.children:
                    if(children.pos_ == 'VERB'):
                        aim_token.append(children)
        
        
            
        #DEONTIC
        #for deontic first check aux from root_aux else check from aim else from keywords and secondary keywords
        for token in root_aux:
            for children in token.children:
                if((children.dep_ == 'aux') and children.pos_ == 'AUX'):
                    deontic_token.append(children)
                    break
            if(len(deontic_token) > 0):
                break

        if(len(deontic_token) == 0):
            for token in aim_token:
                for child in token.children:
                    if(child.dep_ =='aux' and child.pos_ == 'AUX'):
                        deontic_token.append(child)
                        break
                if(len(deontic_token)>0):
                    break

        #if no such deontic found then search in keywords and later in secondary keywords
        if(len(deontic_token) == 0):
            for keyword in keywords:
                for token in doc:
                    if(keyword in token.text and token.pos_ =="AUX"):
                        deontic_token.append(token)
                        break
                if(len(deontic_token)>0):
                    break

        if(len(deontic_token) == 0):
            for keyword in secondary_keywords:
                for token in doc:
                    if(keyword in token.text and token.pos_ =="AUX"):
                        deontic_token.append(token)
                        break
                if(len(deontic_token)>0):
                    break
                    
        #ATTRIBUTE
        for token in root_aux:
            for entity in token.children:
                if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                    attr_token.append(entity)

        for token in aim_token:
            for entity in token.children:
                if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                    attr_token.append(entity)

        #else check for nsubpass
        if(len(attr_token) == 0):
            for token in aim_token:
                for entity in token.children:
                    if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                        attr_token.append(entity)

        #else if empty then check in entire sentence
        if(len(attr_token) == 0):
            for token in doc:
                for entity in token.children:
                    if(entity.dep_ =='nsubj' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                        attr_token.append(entity)
                        break
                if(len(attr_token) > 0):
                    break

        if(len(attr_token) == 0):
            for token in doc:
                for entity in token.children:
                    if(entity.dep_ =='nsubjpass' and (entity.pos_ == "PROPN" or entity.pos_ == "NOUN" or entity.pos_ == "PRON")):
                        attr_token.append(entity)
                        break
                if(len(attr_token) > 0):
                    break
        
        if(len(aim_token) == 0):
            for token in attr_token:
                for children in token.children:
                    if(children.pos_ == 'VERB'):
                        aim_token.append(children)
                        break
                if(len(aim_token)>0):
                    break
                
    elif(case==3 or case==4):
        entity_set = []
        #trying to extract the most common case - a verb with nsubj AND aux both OR try heuristic1
        for token  in doc:
            if(token.pos_ == 'VERB'):
                child_dependency = [child.dep_ for child in token.children]
                str1 = "aux"
                str2 = "nsubj"
                if((str1 in child_dependency) and (str2 in child_dependency)):
                    req_token = []
                    req_token.append(token)
                    for dependencies in token.children:
                        if(dependencies.dep_ == 'aux'):
                            req_token.append(dependencies)
                            break
                    for dependencies in token.children:
                        if(dependencies.dep_ == 'nsubj'):
                            req_token.append(dependencies)
                            break
                    entity_set.append(req_token)
                    
        for token_pair in entity_set:
            for token in token_pair:
                if(token.pos_=='VERB'):
                    aim_token.append(token)
                if(token.dep_=='aux'):
                    attr_token.append(token)
                if(token.dep_=='nsubj'):
                    deontic_token.append(token)
                    
    return aim_token,attr_token,deontic_token

In [36]:
aim_token, attr_token, deontic_token = tokenise(case,doc,root_verb,root_aux)
print(aim_token)
print(attr_token)
print(deontic_token)

[deemed]
[wages]
[shall]


In [37]:
import openpyxl

## Automating responses for all sentences


In [38]:
import openpyxl

# Open the workbook
workbook = openpyxl.load_workbook('new_dataset.xlsx')

# Select the sheet
sheet = workbook['Sheet1']

for i, cell in enumerate(sheet['A']):
    sentence = cell.value
    doc = nlp(sentence)
    
    # Merge noun phrases into one token.
    doc = merge_phrases(doc)
    # Attach punctuation to tokens
    doc = merge_punct(doc)
    
    case,root_verb,root_aux = find_case(doc)
    
    aim_token, attr_token, deontic_token = tokenise(case,doc,root_verb,root_aux)
    
    print(i+1)
    print(sentence)
    sheet.cell(row=i+1, column=2).value =str(aim_token)
    sheet.cell(row=i+1, column=3).value = str(attr_token)
    sheet.cell(row=i+1, column=4).value = str(deontic_token)
    
    
    
    
# Save the worksheet
workbook.save('worksheet.xlsx')



 Case = 2
1
Where immediately before the issue of a notification under section 5 fixing or revising the minimum rates of wages in respect of any scheduled employment, wages at a rate higher than the rate so fixed or revised, were payable either by contract or agreement, or, under any other law for the time being in force, then, notwithstanding anything contained in this Act, wages at such higher rate shall be payable to the employees in such scheduled employment and the wages so payable shall be deemed to be the minimum wages for the purposes of this Act.

 Case = 1
2
Where in respect of an industrial dispute relating to the rates of wages payable to any of the employees employed in a scheduled employment, any proceeding is pending before a Tribunal or National Tribunal under the Industrial Disputes Act, 1947 (24 of 1947) or before any like authority under any other law for the time being in force, or an award made by any Tribunal, National Tribunal or such authority is in operation, 


 Case = 1
21
Where for any accounting year, the allocable surplus exceeds the amount of maximum bonus payable to the employees in the establishment under section 11, then, the excess shall, subject to a limit of twenty per cent. of the total salary or wage of the employees employed in the establishment in that accounting year, be carried forward for being set on in the succeeding accounting year and so on up to and inclusive of the fourth accounting year to be utilized for the purpose of payment of bonus in the manner illustrated in the Fourth Schedule

 Case = 1
22
Where for any accounting year, there is no available surplus or the allocable surplus in respect of that year falls short of the amount of minimum bonus payable to the employees in the establishment under section 10, and there is no amount of sufficient amount carried forward and set on under sub-section (1) which could be utilized for the purpose of payment of the minimum bonus, then, such minimum amount or the deficiency


 Case = 1
39
When the said authority finds that the accounts of such employer of such employer have not been audited by any such auditor and it is of opinion that an audit of the accounts of such employer is necessary for deciding the question referred to it, then, it may, by order direct the employer to get his accounts audited within such time as may be specified in the direction or within such further time as it may allow by such auditor or auditors as it thinks fit and thereupon the employer shall comply with such direction.

 Case = 1
40
Where an employer fails to get the accounts audited under sub-section (2) the said authority may, without prejudice to the provisions of section 28, get the accounts audited by such auditor or auditors as it thinks fit.

 Case = 1
41
The expenses of and incidental to, any audit under sub-section (3) (including the remunerating of the auditor or auditors) shall be determined by the said authority (which determination shall be final) and paid by th


 Case = 1
72
Every worker shall be entitled to eight days' casual leave with vvages in every calendar year which shall be credited into the account of the worker on a quarterly basis.

 Case = 1
73
Every worker who has worked for a period of two hundred and forty days or more in a shop or establishment during a calendar year shall be allowed during the subsequent calendar year, leave with wages for a number of days calculated at the rate of one day for every twenty days of work performed by him during the previous calendar year.

 Case = 1
74
Where the employer refuses to sanction the leave due when applied fifteen days in advance, then the worker shall have a right to encash the leave in excess of forty-five days:

 Case = 1
75
Every employer shall make effective arrangements to provide and maintain at suitable points conveniently situated for all persons employed in the shop or establishment, a sufficient supply of wholesome drinking water.

 Case = 1
76
The State Government may, by


 Case = 1
113
The controlling authority shall, after due inquiry and after giving the parties to the dispute a reasonable opportunity of being heard, determine the matter or matters in dispute and if, as a result of such inquiry any amount is found to be payable to the employee, the controlling authority shall direct the employer to pay such amount or, as the case may be, such amount as reduced by the amount already deposited by the employer.

 Case = 1
114
The controlling authority shall pay the amount deposited, including the excess amount, if any, deposited by the employer, to the person entitled thereto.

 Case = 1
115
The appropriate Government may, by notification, appoint as many Inspectors, as it deems fit, for the purposes of this Act.

 Case = 1
116
The appropriate Government may, by general or special order, define the area to which the authority of an Inspector so appointed shall extend and where two or more Inspectors are appointed for the same area, also provide, by such

### Finding accuracies for the given model

In [55]:
scores_sheet = pd.read_csv('metrics.csv', header=None)
scores_sheet

,0,1,2,3,4,5,6
0,Where immediately before the issue of a notifi...,[deemed],[wages],[shall],wages,deemed,shall
1,Where in respect of an industrial dispute rela...,[apply],[the minimum rates],[shall],employees,apply,shall
2,"At the time of hearing, the authority may dire...",[direct],[the authority],[may],authority,direct,may
3,The authority referred to in sub-section (6) m...,"[allow, referred, preferred]",[The authority],"[may,]",authority,referred,may
4,"Finality of orders constituting Board, Committ...","[called, called]","[No order, any Board,]",[shall],State Government or Chairman or any person,called,shall
...,...,...,...,...,...,...,...
117,No court inferior to that of a Metropolitan Ma...,[try],[No court],[shall],no court,try,shall
118,No suitor other legal proceeding shall lie aga...,[lie],[No suitor],[shall],no suitor,lie,shall
119,The provisions of this Act or any rule made th...,[have],[The provisions],[shall],the provisions,have,shall
120,"The appropriate Government may, by notificatio...",[make],[The appropriate Government],"[may,]",the appropriate government,make,may


In [56]:
def process_element(element):
    # Remove square brackets
    element = element.strip('[]')

    # Split the string on comma and remove extra whitespace
    element_list = [item.strip() for item in element.split(',')]

    return element_list



In [57]:
# Apply the function to the column using apply()
scores_sheet[1] = scores_sheet[1].apply(process_element)
scores_sheet[2] = scores_sheet[2].apply(process_element)
scores_sheet[3] = scores_sheet[3].apply(process_element)


In [58]:
scores_sheet.iloc[0,6]

'shall'

In [59]:
def split_string(element):
    split_list = re.split(r'\bor\b|\band\b', element)
    print(split_list)
    split_list = [item.strip() for item in split_list]
    split_list = list(filter(None, split_list))
    return split_list

In [60]:
scores_sheet[4] = scores_sheet[4].apply(split_string)
scores_sheet[5] = scores_sheet[5].apply(split_string)
scores_sheet[6] = scores_sheet[6].apply(split_string)

['wages']
['employees']
['authority']
['authority']
['State Government ', ' Chairman ', ' any person']
['Chief Inspector ', ' State Government']
['authority']
['Whoever']
['Any person']
['State Government']
['Appropriate Government']
['the reference']
['the reference']
['employee']
['employer ', ' employee']
['employee ', ' employer']
['amount']
['employee']
['employee']
['employees']
['employees']
['principle']
['amount']
['employees']
['employer']
['employer']
['employee']
['provisions']
['employee ', ' any other person']
['application']
['application']
['public sector']
['State ', ' authority']
['company ', ' trade union']
['authority']
['employees']
['authority']
['authority']
['authority']
['The expenses']
['Every employer']
['The appropriate Government']
['Every Inspector']
['Any person']
['Nothing']
['the person']
['nothing']
['an offence']
['No court']
['No court']
['No suit, prosecution ', ' other legal proceeding']
['the provisions']
['Nothing']
['the appropriate Government']

In [61]:
scores_sheet

,0,1,2,3,4,5,6
0,Where immediately before the issue of a notifi...,[deemed],[wages],[shall],[wages],[deemed],[shall]
1,Where in respect of an industrial dispute rela...,[apply],[the minimum rates],[shall],[employees],[apply],[shall]
2,"At the time of hearing, the authority may dire...",[direct],[the authority],[may],[authority],[direct],[may]
3,The authority referred to in sub-section (6) m...,"[allow, referred, preferred]",[The authority],"[may, ]",[authority],[referred],[may]
4,"Finality of orders constituting Board, Committ...","[called, called]","[No order, any Board, ]",[shall],"[State Government, Chairman, any person]",[called],[shall]
...,...,...,...,...,...,...,...
117,No court inferior to that of a Metropolitan Ma...,[try],[No court],[shall],[no court],[try],[shall]
118,No suitor other legal proceeding shall lie aga...,[lie],[No suitor],[shall],[no suitor],[lie],[shall]
119,The provisions of this Act or any rule made th...,[have],[The provisions],[shall],[the provisions],[have],[shall]
120,"The appropriate Government may, by notificatio...",[make],[The appropriate Government],"[may, ]",[the appropriate government],[make],[may]


In [62]:

def calculate_accuracy(df,c1,c2):
    count = 0
    for index, row in df.iterrows():
        
        list_a = row[c1]
        list_b = row[c2]
        flag = False
    # Check if any element from List A is a substring of List B
        for a in list_a:
            for b in list_b:
                if a in b and (flag ==False):
                    flag = True
                    count += 1  # Return 1 if a match is found

    # Check if any element from List B is a substring of List A
        for b in list_b:
            for a in list_a:
                if b in a and (flag ==False):
                    count += 1  # Return 1 if a match is found
                    flag == True
        if(flag==False):
            print(index, row[0])
            print("\n")
    return count/len(df)*100

In [81]:
def convert_to_lowercase(value):
    if isinstance(value, str):
        return value.lower()
    elif isinstance(value, list):
        return [v.lower() for v in value]
    else:
        return value

# Apply the conversion function to the entire DataFrame using applymap()
scores_sheet = scores_sheet.applymap(convert_to_lowercase)

In [82]:
accuracy_deontic = calculate_accuracy(scores_sheet,3,6)
print("\n\n\n")
print(accuracy_deontic)

46 provided that nothing contained in this sub-section shall render any such person liable to any punishment if he proves that the offence was committed without his knowledge or that he exercised all; due diligence to prevent the commission of such offence
53 if the appropriate government, having regard to the financial position and other relevant circumstances of any establishment or class of establishment, is of opinion that it will not be in public interest to apply all or any of the provisions of this act thereto, it may, by notification in the official gazette, exempt for such period as maybe specified therein and subject to such conditions as it may think fit to impose, such establishment or class of establishment from all or any of the provisions of this act.
77 whoever contravenes the provisions of this act or the mles made thereunder shall be punishable with fine which may extend to two lakh rupees and in the case of a continuing contravention, \virh a:1 additional fine which 

In [83]:
accuracy_attr = calculate_accuracy(scores_sheet,2,4)
print("\n\n\n")
print(accuracy_attr)

1 where in respect of an industrial dispute relating to the rates of wages payable to any of the employees employed in a scheduled employment, any proceeding is pending before a tribunal or national tribunal under the industrial disputes act, 1947 (24 of 1947) or before any like authority under any other law for the time being in force, or an award made by any tribunal, national tribunal or such authority is in operation, and a notification fixing or revising the minimum rates of wages in respect of the scheduled employment is issued during the pendency of such proceeding or the operation of the award, then, notwithstanding anything contained in this act, the minimum rates of wages so fixed or so revised shall not apply to those employees during the period in which the proceeding is pending and the award made therein is in operation or, as the case may be, where the notification is issued during the period of operation of an award, during that period; and where such proceeding or award

In [84]:
accuracy_aim = calculate_accuracy(scores_sheet,1,5)
print("\n\n\n")
print(accuracy_aim)

26 where in any accounting year, an employee is found guilty of misconduct causing financial loss to the employer, then, it shall be lawful for the employer to deduct the amount of loss from the amount of bonus payable by him to the employee under this act in respect of that accounting year only and the employee shall be entitled to receive the balance, if any
54 every rule made under this section shall be laid as soon as may be after it is made, before each house of parliament while it is in session for a total period of thirty days, which may be comprised in one session [or in two or more successive sessions], and if before the expiry of the session [immediately following the session or the successive sessions aforesaid], both houses agree in making any modification in the rule or both houses agree that the rule should not be made, the rule shall thereafter have effect only in such modified form or be of no effect, as the case may be; so, however, that any such modification or annulm

### Alternate heuristic

Find all the VERBS in the sentences and return each of them as an AIM

In [85]:
print(accuracy_attr)
print(accuracy_deontic)
print(accuracy_aim)

73.77049180327869
94.26229508196722
93.44262295081968
